<a href="https://colab.research.google.com/github/ykitaguchi77/manipulate_CSV/blob/master/Extract_specific_disease_files_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 特定の疾患の患者の画像ファイルを、眼位写真Newから抜き出す

In [33]:
import pandas as pd

# shift_jis での読み込みを試みる
try:
    df = pd.read_csv("F:\●DiseaseInfo_all_connected_20230907.csv", encoding="shift_jis", header=None)
except UnicodeDecodeError:
    # cp932 での読み込みを試みる
    df = pd.read_csv("F:\●DiseaseInfo_all_connected_20230907.csv", encoding="cp932", header=None)

# 「眼窩腫瘍」あるいは「涙腺腫瘍」が含まれている行を抜き出す
filtered_df = df[df.apply(lambda row: row.astype(str).str.contains('眼窩腫瘍|涙腺腫瘍').any(), axis=1)]

filtered_df

filtered_df.to_csv(r"D:\ダウンロード\filtered_diseases_眼窩腫瘍_涙腺腫瘍.csv", index=False, header=0, encoding="shift_jis")



C:\Users\ykita\AppData\Local\Temp\ipykernel_25044\1381484613.py:5: DtypeWarning: Columns (24,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,69,70,82,84,85,86,87,88,89,90,91,98,100,101,105,112,114,115,116,117,118) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("F:\●DiseaseInfo_all_connected_20230907.csv", encoding="shift_jis", header=None)


In [34]:
# C列 (2番目のカラム) のデータをリストに変換
patient_number_list = filtered_df.iloc[:, 2].tolist()
patient_number_list


[14,
 55,
 144,
 211,
 300,
 332,
 380,
 421,
 566,
 613,
 675,
 676,
 752,
 772,
 773,
 851,
 979,
 981,
 1035,
 1062,
 1339,
 1409,
 1436,
 1670,
 1695,
 1778,
 1811,
 1851,
 1860,
 1965,
 2036,
 2040,
 2132,
 2194,
 2253,
 2309,
 2310,
 2391,
 2396,
 2420,
 2434,
 2720,
 2749,
 2852,
 2884,
 2894,
 2909,
 2932,
 2972,
 2997,
 3015,
 3083,
 3100,
 3209,
 3280,
 3286,
 3319,
 3474,
 3516,
 3524,
 3598,
 3706,
 3753,
 3758,
 3883,
 3896,
 3953,
 3955,
 3965,
 3990,
 4007,
 4021,
 4049,
 4064,
 4082,
 4109,
 4207,
 4255,
 4270,
 4274,
 4318,
 4324,
 4351,
 4446,
 4602,
 4684,
 4724,
 4726,
 4736,
 4765,
 4777,
 4789,
 4825,
 4831,
 4853,
 4889,
 4896,
 4973,
 4978,
 4982,
 5013,
 5075,
 5085,
 5148,
 5250,
 5262,
 5305,
 5319,
 5330,
 5334,
 5346,
 5436,
 5449,
 5458,
 5555,
 5679,
 5694,
 5708,
 5748,
 5799,
 5832,
 5860,
 5877,
 5879,
 5890,
 5902,
 5913,
 5922,
 5928,
 6169,
 6191,
 6197,
 6213,
 6231,
 6368,
 6474,
 6484,
 6530,
 6539,
 6544,
 6555,
 6573,
 6582,
 6688,
 6853,
 6996

In [35]:
"""
特定の数字リストに基づいて眼位写真NEW内のファイルをフィルタリングし、マッチするファイルのフルパスをリストに保存

"""

import os
import re

# 1. Create an empty list
resulting_files = []

# 2. Iterate over all files in the specified directory
# Note: This code cannot be run in this environment, but is provided for reference.
for filename in os.listdir("F:\\眼位写真NEW"):
    # 3. Extract the number from the basename
    base_name = os.path.basename(filename)
    match = re.match(r"^\d+", base_name)
    if match:
        number = int(match.group(0))
        # 4. Check if the number is in the specified list
        if number in patient_number_list:
            resulting_files.append(os.path.join("F:\\眼位写真NEW", filename))

# Output the resulting_files list (commented out since we cannot run the os.listdir in this environment)
resulting_files


['F:\\眼位写真NEW\\1035-20151209-74-122806_15b9ca9c723e6758b94b6bb27a655cb1564266926033d552031a3069d78133c5.jpg',
 'F:\\眼位写真NEW\\1035-20160219-74-134117_1ffc475571cc0b0a277c025999c513ddd69a77b96fc7034c140f70c6af3c2018.jpg',
 'F:\\眼位写真NEW\\1062-20180404-80-103935_9192f0dcaee87f1f8071387fac79037b7a40212ac86a25bd70944fcbf7beae70.jpg',
 'F:\\眼位写真NEW\\1062-20180404-80-133851_08db1dfe18960bc5e785256a12f514a56ff968040dde8af417fa7a036835e8fa.jpg',
 'F:\\眼位写真NEW\\1062-20180404-80-133851_3e18fda968b32ddf462c3d2a8f615d63f48e4954e4d3526bf8b590e59b9c5bb2.jpg',
 'F:\\眼位写真NEW\\1062-20180404-80-133851_5afff5640cf3bbf0f707b295a3390bf487cd6fd5928df028a79c61d6bf0e7c18.jpg',
 'F:\\眼位写真NEW\\1062-20180404-80-133851_a96a29183ef9ca0d801553b03d855a44a5738cc20fa0ff03d908206ffc42f04b.jpg',
 'F:\\眼位写真NEW\\1062-20180404-80-133851_d0f358af5d6d7ffff52d815a8616161f72f120a8e92de37063503d70341ed64c.jpg',
 'F:\\眼位写真NEW\\1062-20180404-80-133851_d7a662dba3e6605d70af163904240e28cd4e210bc9e6d4bbbe7346b0cea862ad.jpg',
 'F:\\眼位写真

In [ ]:
def extract_dates_and_years(file_paths):
    # 日付を抜き出す
    dates = []
    for path in file_paths:
        date_match = re.search(r'(\d{8})', path)
        if date_match:
            dates.append(date_match.group(1))

    # 日付から年を抜き出す
    years = [date[:4] for date in dates]

    return dates, years

def filter_files_by_year(file_paths, start_year):
    _, years = extract_dates_and_years(file_paths)

    # Filtering files based on the year
    filtered_files = [file for file, year in zip(file_paths, years) if int(year) >= start_year]

    return filtered_files

# Filtering the resulting_files list for files from 2017 onwards
filtered_files_2017_onwards = filter_files_by_year(resulting_files, 2017)
filtered_files_2017_onwards


In [38]:
len(filtered_files_2017_onwards)

1533

In [19]:
from tqdm import tqdm

destination_folder = "F:\\眼位写真\\眼位写真_眼窩涙腺腫瘍"

# If the folder already exists, delete it
if os.path.exists(destination_folder):
    shutil.rmtree(destination_folder)
os.makedirs(destination_folder)

# Copy files to the destination folder with tqdm progress bar
for file_path in tqdm(filtered_files_2017_onwards, desc="Copying files"):
    shutil.copy(file_path, destination_folder)

Copying files: 100%|███████████████████████████████████████████████████████████████| 1533/1533 [00:20<00:00, 74.83it/s]
